In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, PredefinedSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [2]:
# train/test set creation

df = pd.read_json("../data/postdatalinesvectors.json",orient='records',lines=True)
X = df[[str(i) for i in range(300)]]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=101)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
# male/female train/test set creation

df = pd.read_json("../data/postdatalinesvectors.json",orient='records',lines=True)

df_male = df[df['gender'] == 'm']
df_female = df[df['gender'] == 'f']

# create male split
Xm = df_male[[str(i) for i in range(300)]]
ym = df_male['label']
X_trainm, X_testm, y_trainm, y_testm = train_test_split(Xm,ym, test_size=0.2, random_state=101)
# create female split
Xf = df_female[[str(i) for i in range(300)]]
yf = df_female['label']
X_trainf, X_testf, y_trainf, y_testf = train_test_split(Xf,yf, test_size=0.2, random_state=101)
# combine splits
X_train = pd.concat([X_trainm,X_trainf])
y_train = pd.concat([y_trainm,y_trainf])

X_test = pd.concat([X_testm,X_testf])
y_test = pd.concat([y_testm,y_testf])

# Scaling for SVM
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_trainm = scaler.fit_transform(X_trainm)
X_trainf = scaler.fit_transform(X_trainf)

X_test = scaler.transform(X_test)
X_testm = scaler.transform(X_testm)
X_testf = scaler.transform(X_testf)

In [4]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

model = SVC(random_state=0, probability=True, class_weight='balanced', C=1, gamma=0.001, kernel='rbf')
model.fit(X_train,y_train)


def prediction_measures(y_test,X_test,confusion=False):
    model_pred = model.predict(X_test)
    performance = classification_report(y_test, model_pred,output_dict=True)
    # display confusion matrix
    if confusion == True:
        confusionmatrix = confusion_matrix(y_test,model_pred)
        cm_display = ConfusionMatrixDisplay(confusion_matrix = confusionmatrix, display_labels = [False, True]) 
        cm_display.plot()
        plt.show()
    return performance['depression'],performance['normal'],performance['accuracy']


### Full set

In [5]:
prediction_measures(y_test,X_test)

({'precision': 0.9367088607594937,
  'recall': 0.8809523809523809,
  'f1-score': 0.9079754601226993,
  'support': 84.0},
 {'precision': 0.8780487804878049,
  'recall': 0.935064935064935,
  'f1-score': 0.9056603773584906,
  'support': 77.0},
 0.906832298136646)

### Male set

In [6]:
# Performance measures male
prediction_measures(y_testm,X_testm)

({'precision': 0.9615384615384616,
  'recall': 0.9259259259259259,
  'f1-score': 0.9433962264150944,
  'support': 27.0},
 {'precision': 0.9285714285714286,
  'recall': 0.9629629629629629,
  'f1-score': 0.9454545454545454,
  'support': 27.0},
 0.9444444444444444)

### Female set

In [7]:
# Performance measures female
prediction_measures(y_testf,X_testf)

({'precision': 0.9245283018867925,
  'recall': 0.8596491228070176,
  'f1-score': 0.8909090909090909,
  'support': 57.0},
 {'precision': 0.8518518518518519,
  'recall': 0.92,
  'f1-score': 0.8846153846153846,
  'support': 50.0},
 0.8878504672897196)

### Statistical parity

In [8]:
# predictions
pred_male = model.predict(X_testm)
pred_female = model.predict(X_testf)

def percentage_depression(pred):
    return np.sum(pred == 'depression') / len(pred)

def statistical_parity(predsensitive,predother):
    return percentage_depression(predsensitive)/percentage_depression(predother)

statistical_parity(pred_female,pred_male)

1.0287562904385334

### Equal opportunity

In [9]:
# predictions
pred_male = model.predict(X_testm)
pred_female = model.predict(X_testf)

def true_positives(pred, true):
    return np.sum((pred == 'depression') & (true == "depression"))

def true_negatives(pred, true):
    return np.sum((pred == 'normal') & (true == "normal"))

def true_positive_rate(pred, true):
    return true_positives(pred,true)/(true_positives(pred,true)+true_negatives(pred,true))

def equal_opportunity(predsensitive, truesensitve, predother, trueother):
    return true_positive_rate(predsensitive,truesensitve)/true_positive_rate(predother,trueother)




equal_opportunity(pred_female,y_testf,pred_male,y_testm)



1.0522105263157897

## Equalised odds

In [10]:
# predictions
pred_male = model.predict(X_testm)
pred_female = model.predict(X_testf)

def true_negative_rate(pred, true):
    return true_negatives(pred,true)/(true_positives(pred,true)+true_negatives(pred,true))

def equalised_odds(predsensitive, truesensitve, predother, trueother):
    return (true_positive_rate(predsensitive,truesensitve)+true_negative_rate(predsensitive,truesensitve))/(true_positive_rate(predother,trueother)+true_negative_rate(predsensitive,truesensitve))

equalised_odds(pred_female,y_testf,pred_male,y_testm)

1.0262656216903199

## Equal accuracy

In [11]:
# predictions
pred_male = model.predict(X_testm)
pred_female = model.predict(X_testf)

def equal_accuracy(predsensitive, truesensitve, predother, trueother):
    return accuracy_score(predsensitive,truesensitve)/accuracy_score(predother,trueother)

equal_accuracy(pred_female,y_testf,pred_male,y_testm)

0.9400769653655855

### Performance measurement functions

In [49]:
# Performance measures

def prediction_measures(y_test,y_pred,confusion=False):
    performance = classification_report(y_test,y_pred,output_dict=True)
    # display confusion matrix
    if confusion == True:
        confusionmatrix = confusion_matrix(y_test,y_pred)
        cm_display = ConfusionMatrixDisplay(confusion_matrix = confusionmatrix, display_labels = [False, True]) 
        cm_display.plot()
        plt.show()
    return performance['depression'],performance['normal'],performance['accuracy']

# Support measures

def percentage_depression(pred):
    return np.sum(pred == 'depression') / len(pred)

def true_positives(pred, true):
    return np.sum((pred == 'depression') & (true == "depression"))

def true_negatives(pred, true):
    return np.sum((pred == 'normal') & (true == "normal"))

def true_positive_rate(pred, true):
    return true_positives(pred,true)/(true_positives(pred,true)+true_negatives(pred,true))

def true_negative_rate(pred, true):
    return true_negatives(pred,true)/(true_positives(pred,true)+true_negatives(pred,true))

# Complete measures

def statistical_parity(predsensitive,predother):
    return percentage_depression(predsensitive)/percentage_depression(predother)

def equal_opportunity(predsensitive, truesensitive, predother, trueother):
    return true_positive_rate(predsensitive,truesensitive)/true_positive_rate(predother,trueother)

def equalised_odds(predsensitive, truesensitive, predother, trueother):
    return (true_positive_rate(predsensitive,truesensitive)+true_negative_rate(predsensitive,truesensitive))/(true_positive_rate(predother,trueother)+true_negative_rate(predsensitive,truesensitive))

def equal_accuracy(predsensitive, truesensitive, predother, trueother):
    return accuracy_score(predsensitive,truesensitive)/accuracy_score(predother,trueother)

# All measurements

def all_measures(predsensitive, truesensitive, predother, trueother, name='test',single=False):
    score_dict = {}
    score_dict['test'] = name
    depression_performance, normal_performance ,accuracy = prediction_measures(predsensitive,truesensitive)
    score_dict['depression0precision'] = depression_performance['precision']
    score_dict['depression0recall'] = depression_performance['recall']
    score_dict['depression0f1'] = depression_performance['f1-score']
    score_dict['depression0support'] = depression_performance['support']
    score_dict['normal0precision'] = normal_performance['precision']
    score_dict['normal0recall'] = normal_performance['recall']
    score_dict['normal0f1'] = normal_performance['f1-score']
    score_dict['normal0support'] = normal_performance['support']
    score_dict['accuracy0'] = accuracy
    if single == False:
        depression_performance, normal_performance ,accuracy = prediction_measures(predother,trueother)
        score_dict['depression1precision'] = depression_performance['precision']
        score_dict['depression1recall'] = depression_performance['recall']
        score_dict['depression1f1'] = depression_performance['f1-score']
        score_dict['depression1support'] = depression_performance['support']
        score_dict['normal1precision'] = normal_performance['precision']
        score_dict['normal1recall'] = normal_performance['recall']
        score_dict['normal1f1'] = normal_performance['f1-score']
        score_dict['normal1support'] = normal_performance['recall']
        score_dict['accuracy1'] = accuracy
        score_dict['statisticalParity01'] = statistical_parity(predsensitive,predother)
        score_dict['equalOpportunity01'] = equal_opportunity(predsensitive, truesensitive, predother, trueother)
        score_dict['equalisedOdds'] = equalised_odds(predsensitive, truesensitive, predother, trueother)
        score_dict['equalAccuracy'] = equal_accuracy(predsensitive, truesensitive, predother, trueother)
    return score_dict

In [52]:
df = pd.DataFrame(all_measures(pred_female,y_testf,pred_male,y_testm),index=[0])

df

,test,depression0precision,depression0recall,depression0f1,depression0support,normal0precision,normal0recall,normal0f1,normal0support,accuracy0,...,depression1support,normal1precision,normal1recall,normal1f1,normal1support,accuracy1,statisticalParity01,equalOpportunity01,equalisedOdds,equalAccuracy
0,test,0.859649,0.924528,0.890909,53.0,0.92,0.851852,0.884615,54.0,0.88785,...,26.0,0.962963,0.928571,0.945455,0.928571,0.944444,1.028756,1.052211,1.026266,0.940077
